<a href="https://colab.research.google.com/github/natsakh/IAD/blob/main/Pr_7/7_7_HF_Fine_tuning_DistilBERT_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install transformers

In [2]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

os.environ["WANDB_DISABLED"] = "true"

Device: cuda


In [13]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")


Мінімальний набір для fine-tuning через Trainer

Для тренування DistilBERT потрібні:

✔ AutoTokenizer - Щоб токенізувати текст

✔ AutoModelForSequenceClassification - Модель з classifier-головою

✔ DataCollatorWithPadding - Автоматичний батч-паддінг

✔ TrainingArguments - Налаштування тренування

✔ Trainer - Запуск навчання

In [4]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

#DistilBERT для донавчання на IMDB

Тепер беремо модель: distilbert-base-uncased (без готової “голови” на SST-2), і будемо її донавчати на 2 класи IMDB.

In [14]:
#Токенайзер + модель
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,          # IMDB: 0 = negative, 1 = positive
)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#Функція токенізації
def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding=False,
        truncation=True,
        max_length=256,
    )


In [15]:
# Напр., по 4000 прикладів на train / test
small_train = raw_datasets["train"].shuffle(seed=42).select(range(4000))
small_test  = raw_datasets["test"].shuffle(seed=42).select(range(1000))

tokenized_train = small_train.map(tokenize_batch, batched=True)
tokenized_test  = small_test.map(tokenize_batch,  batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [10]:
batch_size= 16
training_args = TrainingArguments(
    output_dir="distilbert-imdb",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-3908350211.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.357800,0.287111,0.870000
2,0.163100,0.304708,0.876000


TrainOutput(global_step=500, training_loss=0.3013623752593994, metrics={'train_runtime': 208.0246, 'train_samples_per_second': 38.457, 'train_steps_per_second': 2.404, 'total_flos': 529869594624000.0, 'train_loss': 0.3013623752593994, 'epoch': 2.0})